In [ ]:
import os
import json
import plotly.graph_objs as go
import plotly.io as pio
import networkx as nx

In [ ]:
script_path = os.path.join(*[os.path.abspath(""), "src", "py", "scraping", "yahoo"])
index_path = os.path.join(script_path, "index.json")

In [ ]:
index = json.load(open(index_path, "r"))
print(f"Symbols: {len(index['symbols'])}")

In [ ]:
symbols = index["symbols"]
G = nx.Graph()

# Add nodes
for symbol in symbols:
	G.add_node(symbol)

# Add edges
for symbol in symbols:
	symbol_obj = index["symbols"][symbol]
	similar = symbol_obj["similar"]
	recommended = symbol_obj["recommended"]
	for s in similar:
		G.add_edge(symbol, s)
	# for r in recommended:
	# 	G.add_edge(symbol, r)



In [ ]:
# Defaults
k = 0.5
iterations = 50
seed = 42
# k = 10 / len(G.nodes())
k = 0.5
iterations = 100
pos = nx.spring_layout(G, k=k, iterations=iterations, seed=seed)

edge_trace = go.Scatter(
	x=[],
	y=[],
	line=dict(width=0.5,color='#888'),
	hoverinfo='none',
	mode='lines')

for edge in G.edges():
	x0, y0 = pos[edge[0]]
	x1, y1 = pos[edge[1]]
	edge_trace['x'] += tuple([x0, x1, None])
	edge_trace['y'] += tuple([y0, y1, None])

node_trace = go.Scatter(
	x=[],
	y=[],
	text=[],
	mode='markers+text',
	hoverinfo='text',
	textfont=dict(
		family='Arial',
		size=10,
		color='#000',
		
	),
	marker=dict(
		showscale=True,
		# colorscale options
		# 'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
		# 'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
		# 'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
		colorscale='YlGnBu',
		reversescale=True,
		color=[],
		size=40,
		colorbar=dict(
			thickness=15,
			title='Node Connections',
			xanchor='left',
			titleside='right'
		),
		line=dict(width=2)))

for node in G.nodes():
	x, y = pos[node]
	node_trace['x'] += tuple([x])
	node_trace['y'] += tuple([y])
	node_trace['text'] += tuple([node])
	node_trace['marker']['color'] += tuple([len(G.adj[node])])
	
layout = go.Layout(
	showlegend=False,
	hovermode='closest',
	width=1080,
	height=1080,
	margin=dict(b=20,l=5,r=5,t=40),
	xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
	yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

fig = go.Figure(data=[edge_trace, node_trace], layout=layout)

fig.show()

In [ ]:
img_path = os.path.join(script_path, "relations.png")
pio.write_image(fig, img_path)

In [ ]:
# Sort by number of connections
nodes_sorted = sorted(G.adj.items(), key=lambda x: len(x[1]), reverse=True)
# Print number of connections for each node
for node in nodes_sorted:
	node_title = "not in crawled symbols"
	if node[0] in index["symbols"]:
		node_title = index["symbols"][node[0]]["title"]
	print(f"{node[0]}: {len(node[1])}        ({node_title})")